Intro to cuDF
=======================

Welcome to first cuDF tutorial notebook! This is a short introduction to cuDF, partly modeled after 10 Minutes to Pandas, geared primarily for new users. cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data.

We'll start by introducing the pandas library, and quickly move on cuDF.

In [167]:
import os
import numpy as np
import math
np.random.seed(12)

<a id="pandas"></a>
## Pandas

Data scientists typically work with two types of data: unstructured and structured. Unstructured data often comes in the form of text, images, or videos. Structured data - as the name suggests - comes in a structured form, often represented by a table or CSV. We'll focus the majority of these tutorials on working with structured data.

There exist many tools in the Python ecosystem for working with structured, tabular data but few are as widely used as Pandas. Pandas represents data in a table and allows a data scientist to manipulate the data to perform a number of useful operations such as filtering, transforming, aggregating, merging, visualizing and many more. 

For more information on Pandas, check out the excellent documentation: http://pandas.pydata.org/pandas-docs/stable/

Below we show how to create a Pandas DataFrame, an internal object for representing tabular data.

In [168]:
import pandas as pd; print('Pandas Version:', pd.__version__)


# here we create a Pandas DataFrame with
# two columns named "key" and "value"
df = pd.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

Pandas Version: 0.23.4
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


We can perform many operations on this data. For example, let's say we wanted to sum all values in the in the `value` column. We could accomplish this using the following syntax:

In [169]:
aggregation = df['value'].sum()
print(aggregation)

60.0


<a id="cudf"></a>
## cuDF

Pandas is fantastic for working with small datasets that fit into your system's memory and workflows that are not computationally intense. However, datasets are growing larger and data scientists are working with increasingly complex workloads - the need for accelerated compute arises.

cuDF is a package within the RAPIDS ecosystem that allows data scientists to easily migrate their existing Pandas workflows from CPU to GPU, where computations can leverage the immense parallelization that GPUs provide.

Below, we show how to create a cuDF DataFrame.

In [170]:
import cudf; print('cuDF Version:', cudf.__version__)


# here we create a cuDF DataFrame with
# two columns named "key" and "value"
df = cudf.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

cuDF Version: 0.8.0a1+606.g878f02e
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


As before, we can take this cuDF DataFrame and perform a `sum` operation over the `value` column. The key difference is that any operations we perform using cuDF use the GPU instead of the CPU.

In [171]:
aggregation = df['value'].sum()
print(aggregation)

60.0


Note how the syntax for both creating and manipulating a cuDF DataFrame is identical to the syntax necessary to create and manipulate Pandas DataFrames; the cuDF API is based on the Pandas API. This design choice minimizes the cognitive burden of switching from a CPU based workflow to a GPU based workflow and allows data scientists to focus on solving problems while benefitting from the speed of a GPU!

# DataFrame Basics with cuDF

In the following tutorial, you'll get a chance to familiarize yourself with cuDF. For those of you with experience using pandas, this should look nearly identical.

Along the way you'll notice small exercises. These exercises are designed to help you get a feel for writing the code yourself, but if you get stuck, you can take a look at the solutions.

Portions of this were borrowed from the 10 Minutes to cuDF guide.

Object Creation
---------------

Creating a `cudf.Series`.

In [172]:
s = cudf.Series([1,2,3,None,4])
print(s)

0    1
1    2
2    3
3     
4    4
dtype: int64


Creating a `cudf.DataFrame` by specifying values for each column.

In [173]:
df = cudf.DataFrame([('a', list(range(20))),
('b', list(reversed(range(20)))),
('c', list(range(20)))])
print(df)

   a   b  c
0  0  19  0
1  1  18  1
2  2  17  2
3  3  16  3
4  4  15  4
5  5  14  5
6  6  13  6
7  7  12  7
8  8  11  8
9  9  10  9
[10 more rows]


Creating a `cudf.DataFrame` from a `pd.Dataframe`.

In [174]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

   a    b
0  0  0.1
1  1  0.2
2  2     
3  3  0.3


Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [175]:
print(df.head(2))

   a   b  c
0  0  19  0
1  1  18  1


Sorting by values.

In [176]:
print(df.sort_values(by='b'))

    a  b   c
19  19  0  19
18  18  1  18
17  17  2  17
16  16  3  16
15  15  4  15
14  14  5  14
13  13  6  13
12  12  7  12
11  11  8  11
10  10  9  10
[10 more rows]


Selection
------------

## Getting

Selecting a single column, which initially yields a `cudf.Series` (equivalent to `df.a`).

In [177]:
print(df['a'])

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
[10 more rows]
Name: a, dtype: int64


## Selection by Label

Selecting rows from index 2 to index 5 from columns `a` and `b`.

In [178]:
print(df.loc[2:5, ['a', 'b']])

   a   b
2  2  17
3  3  16
4  4  15
5  5  14


## Selection by Position

Selecting via integers and integer slices, like numpy/pandas.

In [179]:
print(df.iloc[0])

a     0
b    19
c     0
Name: 0, dtype: int64


In [180]:
print(df.iloc[0:3, 0:2])

   a   b
0  0  19
1  1  18
2  2  17


You can also select elements of a `DataFrame` or `Series` with direct index access.

In [181]:
print(df[3:5])

   a   b  c
3  3  16  3
4  4  15  4


In [182]:
print(s[3:5])

3     
4    4
dtype: int64


## Exercise 1

Try to select only the rows at index `4` and `9` from `df`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.iloc[[4,9]])
   </pre>
</details>

## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [183]:
print(df[df.b > 15])

   a   b  c
0  0  19  0
1  1  18  1
2  2  17  2
3  3  16  3


Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [184]:
print(df.query("b == 3"))  

    a  b   c
16  16  3  16


In [185]:
val = 3
df.query("b == @val")

<cudf.DataFrame ncols=3 nrows=1 >

You can also pass local variables to cuDF queries, via the `local_dict` keyword or `@` operator.

In [186]:
cudf_comparator = 3
print(df.query("b == @cudf_comparator"))

    a  b   c
16  16  3  16


Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

## Exercise 2

Try to select only the rows from `df` where the value in column `b` is greater than the vaue in column `c` + 6.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [190]:
print(s.fillna(999))

0      1
1      2
2      3
3    999
4      4
dtype: int64


Operations
------------

## Stats

Calculating descriptive statistics for a `Series`.

In [204]:
s = cudf.Series(np.arange(10)).astype(np.float32)

In [205]:
print(s.mean(), s.var())

4.5 9.166666666666668


In [206]:
print(df.describe())

   stats        a        b        c
0  count     20.0     20.0     20.0
1   mean      9.5      9.5      9.5
2    std  5.91608  5.91608  5.91608
3    min      0.0      0.0      0.0
4    25%     4.75     4.75     4.75
5    50%      9.5      9.5      9.5
6    75%    14.25    14.25    14.25
7    max     19.0     19.0     19.0


## Applymap

Applying functions to a `Series`.

In [207]:
def add_ten(num):
    return num + 10

print(s.applymap(add_ten))

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float32


We can also apply more complicated functions, such as this one.

In [209]:
def complex_math_transform(num):
    return math.cos(num) * 3 / 9

print(s.applymap(complex_math_transform))

0      0.33333334
1      0.18010075
2     -0.13871561
3      -0.3299975
4      -0.2178812
5      0.09455406
6      0.32005677
7      0.25130075
8    -0.048500013
9     -0.30371007
dtype: float32


## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the cuDF and [nvStrings API](https://docs.rapids.ai/api/nvstrings/nightly/) documentation for more information.

In [210]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
print(s.str.lower())

0       a
1       b
2       c
3    aaba
4    baca
5    None
6    caba
7     dog
8     cat
dtype: object


## Exercise 3

Try to convert all the strings to uppercase. Take a look at the nvStrings API documentation linked above if you need some help.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [211]:
s = cudf.Series([1, 2, 3, None, 5])
print(cudf.concat([s, s]))

0    1
1    2
2    3
3     
4    5
0    1
1    2
2    3
3     
4    5
dtype: int64


## Append

Appending values from another `Series` or array-like object.

In [215]:
print(s.append(s))

0    1
1    2
2    3
3     
4    5
0    1
1    2
2    3
3     
4    5
dtype: int64


## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [216]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
print(merged)

   key  vals_a  vals_b
0    a    10.0   100.0
1    c    12.0   101.0
2    e    14.0   102.0
3    b    11.0        
4    d    13.0        


## Exercise 4

Using the DataFrames we created above, try to do an `inner` join with using `merge`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df_a.merge(df_b, on=['key'], how='inner'))
   </pre>
</details>

## Grouping

Like pandas, cuDF supports the [Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) groupby paradigm.

In [217]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

Grouping and then applying the `sum` function to the grouped data.

In [218]:
print(df.groupby('agg_col1').sum())

     a    b    c  agg_col2
agg_col1
0  100   90  100         3
1   90  100   90         4


Grouping and applying statistical functions to specific columns, using `agg`.

In [220]:
print(df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}))

    a     b    c
agg_col1
0  19   9.0  100
1  18  10.0   90


## Exercise 5

We can also group by multiple columns at once, which we call grouping hierarchically. Try to group `df` by `agg_col1` and `agg_col2` and take groupby mean of column `a` and minimum of column `b`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}).to_pandas())
   </pre>
</details>

Time Series
------------


`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [125]:
import datetime as dt

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = np.random.sample(len(date_df))
print(date_df.head())

                     date                value
0 2018-11-20T00:00:00.000   0.3168079006890451
1 2018-11-21T00:00:00.000   0.2417802357140627
2 2018-11-22T00:00:00.000   0.4909549698249829
3 2018-11-23T00:00:00.000   0.5439434095685136
4 2018-11-24T00:00:00.000  0.40545438708501025


## Exercise 6

Try to use `query` to filter `date_df` to only those row with a date before `2018-11-23`. This is a bit trickier than the prior exercises. As a hint, you'll want to explore the `strptime` function from the `datetime` library. We've already imported this in the cell above.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>
    search_date = dt.datetime.strptime('2018-11-23', '%Y-%m-%d')
    print(date_df.query("date &lt= @search_date"))
            </br>
   </pre>
</details>

You can also interact with datetime columns to extract things like the day, hour, minute, and more.

In [276]:
date_df['minute'] = date_df.date.dt.minute
print(date_df.head())

                 date     value  minute  day
0 2019-07-05 00:00:00  0.666495       0    5
1 2019-07-05 00:00:01  0.358797       0    5
2 2019-07-05 00:00:02  0.603502       0    5
3 2019-07-05 00:00:03  0.978165       0    5
4 2019-07-05 00:00:04  0.572789       0    5


Categoricals
------------

`DataFrames` support categorical columns.

In [112]:
pdf = pd.DataFrame({"id":[1,2,3,4,5,6], "grade":['a', 'b', 'b', 'a', 'a', 'e']})
pdf["grade"] = pdf["grade"].astype("category")

gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

   id  grade
0   1      a
1   2      b
2   3      b
3   4      a
4   5      a
5   6      e


Accessing the categories of a column.

In [115]:
gdf.grade.cat.categories

('a', 'b', 'e')

Accessing the underlying code values of each categorical observation.

In [116]:
print(gdf.grade.cat.codes)

0    0
1    1
2    1
3    0
4    0
5    2
dtype: int8


Converting Data Representation
--------------------------------

## Pandas

Converting a cuDF `DataFrame` to a pandas `DataFrame`.

In [117]:
print(df.head().to_pandas())

   a   b  c  agg_col1  agg_col2
0  0  19  0         1         1
1  1  18  1         0         0
2  2  17  2         1         0
3  3  16  3         0         1
4  4  15  4         1         0


## Numpy

Converting a cuDF `DataFrame` to a numpy `ndarray`.

In [118]:
print(df.as_matrix())

[[ 0 19  0  1  1]
 [ 1 18  1  0  0]
 [ 2 17  2  1  0]
 [ 3 16  3  0  1]
 [ 4 15  4  1  0]
 [ 5 14  5  0  0]
 [ 6 13  6  1  1]
 [ 7 12  7  0  0]
 [ 8 11  8  1  0]
 [ 9 10  9  0  1]
 [10  9 10  1  0]
 [11  8 11  0  0]
 [12  7 12  1  1]
 [13  6 13  0  0]
 [14  5 14  1  0]
 [15  4 15  0  1]
 [16  3 16  1  0]
 [17  2 17  0  0]
 [18  1 18  1  1]
 [19  0 19  0  0]]


Converting a cuDF `Series` to a numpy `ndarray`.

In [119]:
print(df['a'].to_array())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Getting Data In/Out
------------------------


## CSV

Writing to a CSV file, by first sending data to a pandas `Dataframe` on the host.

In [121]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_pandas().to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [122]:
df = cudf.read_csv('example_output/foo.csv')
print(df)

   a   b  c  agg_col1  agg_col2
0  0  19  0         1         1
1  1  18  1         0         0
2  2  17  2         1         0
3  3  16  3         0         1
4  4  15  4         1         0
5  5  14  5         0         0
6  6  13  6         1         1
7  7  12  7         0         0
8  8  11  8         1         0
9  9  10  9         0         1
[10 more rows]


That's it! You've got the basics of cuDF down! Let's talk a little bit about the computational performance of cuDF and GPUs.

# Performace

One of the primary reasons to use cuDF over pandas is performance. For some workflows, the GPU can be up to 1000x faster than the CPU. Let's illustrate this by starting with a small example: Creating a DataFrame and calculating the sum of a column.

In [331]:
a = np.random.rand(10000000) # 100 million values

In [332]:
pdf = pd.DataFrame()
cdf = cudf.DataFrame()

In [333]:
%time pdf['a'] = a
%time cdf['a'] = a

CPU times: user 680 ms, sys: 400 ms, total: 1.08 s
Wall time: 1.08 s
CPU times: user 12 ms, sys: 12 ms, total: 24 ms
Wall time: 22.8 ms


In [334]:
%%timeit
pdf['a'].sum()

100 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [335]:
%%timeit
cdf['a'].sum()

1.04 ms ± 36 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Pretty cool! cuDF was about 10x faster in creating the DataFrame, and 100x faster in calculating the sum! This is a pretty small example, though.

### A More Realistic Example: Sensor Data Analytics

To get a more realistic sense of how powerful cuDF and GPUs can be, let's imagine you had a fleet of sensors that collect data every millisecond. These sensors could be measuring pressure, temperature, or something else entirely.

Let's imagine we want to analyze one day's worth of sensor data. We'll assign random values for the sensor `value` to use for this example. We'll start by creating the data, and generating some datetime related features like we learned about in the above tutorial.

In [309]:
%%time

date_df = pd.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

# date_df['day'] = date_df.date.dt.day
date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
# date_df['second'] = date_df.date.dt.second

print(date_df.shape)
date_df.head()

(86400001, 4)
CPU times: user 12.1 s, sys: 5.05 s, total: 17.2 s
Wall time: 17.2 s


Just creating the data took more than 15 seconds! Let's do our analysis. From our sensor data, we want to get the maximum sensor value for each minute. Since we don't want to combine values in the same minute of different hours, we'll need to do a hierarchical groupby.

In [312]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

CPU times: user 3.38 s, sys: 3.27 s, total: 6.64 s
Wall time: 6.64 s


value
hour minute          
0    0       0.999992
     1       0.999983
     2       0.999998
     3       0.999992
     4       0.999984

Six seconds just do do this! Imagine if we had a fleet of sensors, or wanted to get the maximum sensor value every within every second?!

Let's try this in cuDF now, using the GPU DataFrame. We'll run the same code as above.

In [313]:
%%time

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

# date_df['day'] = date_df.date.dt.day
date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
# date_df['second'] = date_df.date.dt.second

print(date_df.shape)
print(date_df.head())

(86400001, 4)
                     date               value  hour  minute
0 2019-07-05T00:00:00.000  0.9612777381745159     0       0
1 2019-07-05T00:00:00.001  0.2388221383735578     0       0
2 2019-07-05T00:00:00.002  0.7504417943077807     0       0
3 2019-07-05T00:00:00.003  0.8961592441252346     0       0
4 2019-07-05T00:00:00.004  0.8194703662291679     0       0
CPU times: user 3.02 s, sys: 2.1 s, total: 5.12 s
Wall time: 5.12 s


Only five seconds to create the data. That's 3x as fast just within creating the dataframe and features. Let's try the same groupby aggregation.

In [315]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

CPU times: user 104 ms, sys: 104 ms, total: 208 ms
Wall time: 208 ms


<cudf.DataFrame ncols=1 nrows=5 >

This should have taken about 200-300 milliseconds. That's 30x faster than pandas, with the same code!

## Exercise 7

Play around with some more pandas and cuDF operations and compare the performance between them. What operation can you find that gives the highest performance ratio? 

You can start a cell with `%%time` to time the cell, or with `%%timeit`, which runs the cell multiple times and gives an average. `%%timeit` gives a more accurate benchmark but takes longer to run.